Updating the metadata files to reflect their true paths on Sherlock

In [1]:
import pandas as pd
import os
import time
# from multiprocessing import Pool

In [2]:
loc = '/oak/stanford/groups/malgeehe/corpora/apo/metadata/lila_meta/'

In [3]:
sher = '/oak/stanford/groups/malgeehe/corpora/apo/ProQuest_AmPeriodicals' # location of zips on sherlock

In [4]:
csvs = [os.path.join(loc,x) for x in os.listdir(loc) if x.endswith('.csv')]

In [5]:
def update_meta(csv):
    start = time.time()
    
    df = pd.read_csv(csv, low_memory = False) # mixed dtypes
    
    # make sherlock zip_paths
    df['archive'] = [os.path.join(sher, os.path.split(x)[1]) for x in df['zip_path']]

    # clarify time
    df['year'] = [int(str(x)[:4]) for x in df['numericpubdate']]
    df['month'] = [int(str(x)[4:6]) for x in df['numericpubdate']]
    df['day'] = [int(str(x)[6:]) for x in df['numericpubdate']]

    # retain columns valuable on sherlock
    cols = ['recordid', 'recordtitle', 'objecttype', 'year', 'month', 'day', 'file_name', 'title',
            'publicationid', 'qualifier',  'archive', 'n_valid_words', 'ocr_acc']

    names = ['record_id', 'record_title', 'record_type', 'year', 'month', 'day', 'file_name', 'pub_title', 
             'pub_id', 'pub_location', 'zip_path', 'record_tokens', 'record_ocr']

    # drop old cols and rename
    df = df[cols]
    df.columns = names

    # make record_id index
    df.set_index('record_id', inplace = True)

    # write output
    out = '/oak/stanford/groups/malgeehe/corpora/apo/metadata/sherlock_meta'
    filename = os.path.split(csv)[1].replace('merge', 'sherlock')
    filepath = os.path.join(out, filename)
    df.to_csv(filepath)
    
    total = round(time.time() - start)
    print('{} in {} seconds'.format(filepath, total))
    return

In [ ]:
for x in csvs:
    update_meta(x)

In [ ]:
# with Pool() as p:
#     files = p.map(update_meta, csvs)